# Lesson 14: WhatsApp Business Co-pilot

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harshit-vibes/lyzr-adk-demo/blob/master/notebooks/14_whatsapp_business_copilot.ipynb)


🔴 **Advanced** · ⏱ **35 min**

---

In production, AI agents often need to operate inside messaging channels rather than APIs.
WhatsApp Business is the world's most-used messaging platform.
This lesson shows how to wire a lyzr-adk agent into the Meta WhatsApp Cloud API architecture —
including a webhook receiver, order lookup tool, and customer session memory.

Every cell in this notebook is **runnable in simulation mode** — no real Meta credentials needed.
The architecture is identical to a production deployment; only the `send_whatsapp_reply` tool
is simulated (it prints instead of calling the Meta API).

## What you'll learn

- Understand the Meta WhatsApp Cloud API webhook architecture
- Build a customer service agent with order lookup tool
- Use `session_id = customer_phone` for per-customer conversation memory
- Simulate an end-to-end WhatsApp conversation (inquiry → order lookup → escalation)
- See the complete production FastAPI webhook receiver code

## Prerequisites

Before running this notebook, you should be familiar with:

- **Lesson 1** — Studio & `agent.run()` basics
- **Lesson 5** — Memory & sessions (`session_id` pattern)
- **Lesson 6** — Custom tools (`agent.add_tool()`, docstrings)
- **Lesson 9** — RAI guardrails (optional but helpful)

You also need your **`LYZR_API_KEY`** set as an environment variable:

```bash
export LYZR_API_KEY="your-api-key-here"
```

> **Note:** No Meta/WhatsApp credentials are required for this notebook.
> All WhatsApp sends are simulated by printing to stdout.

In [ ]:
!pip install lyzr-adk[jupyter] fastapi uvicorn -q

In [ ]:
import os
import uuid
from lyzr import Studio

API_KEY = os.getenv("LYZR_API_KEY", "YOUR_LYZR_API_KEY")
studio = Studio(api_key=API_KEY)
print("Ready!")

## 1. Architecture Overview

A production WhatsApp Business integration has three layers:

| Layer | What it does |
|---|---|
| **Meta Webhook** | Meta sends a POST to your server when a customer messages you |
| **FastAPI receiver** | Your server validates the request and routes the message to your agent |
| **lyzr-adk agent** | Processes the message, calls tools, and replies via the Meta API |

```
Customer → [WhatsApp] → Meta servers → POST /webhook → FastAPI → lyzr-adk agent
                                                                         │
                                                               tool: lookup_order()
                                                               tool: send_whatsapp_reply()
                                                                         │
                                                               Reply → Meta API → Customer
```

In **simulation mode** (this notebook), we skip the HTTP layer and call the agent directly,
exactly as the webhook handler would. The `send_whatsapp_reply` tool prints instead of calling Meta.

## 2. Building the Tools

Two tools power this agent:

1. **`lookup_order`** — queries a simulated order database by order ID
2. **`send_whatsapp_reply`** — in production calls `https://graph.facebook.com/v18.0/{PHONE_NUMBER_ID}/messages`; in simulation mode, prints to stdout

The docstring is critical for both: the agent reads it to decide when and with what arguments to call each tool.

In [ ]:
# Tool 1: Order lookup (simulated database)
ORDERS = {
    "ORD-1001": {"status": "shipped",    "delivery": "2026-02-26", "carrier": "FedEx", "tracking": "FX123456"},
    "ORD-1002": {"status": "processing", "delivery": "2026-02-28", "carrier": "UPS",   "tracking": "UP789012"},
    "ORD-1003": {"status": "delivered",  "delivery": "2026-02-20", "carrier": "DHL",   "tracking": "DH345678"},
}

def lookup_order(order_id: str) -> str:
    """Look up a customer order by order ID and return its current status.

    Returns the shipping status, carrier name, tracking number, and estimated delivery date.
    Use this whenever a customer asks about the status, location, or delivery date of their order.
    Order IDs follow the format ORD-XXXX.
    """
    if order_id.upper() in ORDERS:
        o = ORDERS[order_id.upper()]
        return (
            f"Order {order_id.upper()}: {o['status']} via {o['carrier']} "
            f"(tracking: {o['tracking']}), estimated delivery: {o['delivery']}"
        )
    return f"Order {order_id} not found. Please verify the order ID and try again."


# Tool 2: WhatsApp reply sender (simulated)
# Called by the webhook handler (not the agent) to deliver the agent's response to the customer.
def send_whatsapp_reply(to_number: str, message: str) -> str:
    """Send a WhatsApp reply message to a customer's phone number.

    In production this calls the Meta Cloud API. In simulation it prints the message.
    This function is called by the webhook handler after agent.run() returns a response —
    it is not a tool given to the agent.
    """
    print(f"\n📱 [WhatsApp → {to_number}]\n{message}\n")
    return f"Message delivered to {to_number}"


print("Tools defined: lookup_order, send_whatsapp_reply")

## 3. Creating the WhatsApp Co-pilot Agent

The agent needs:
- A **role** that establishes it as a customer service agent
- The `lookup_order` tool attached via `agent.add_tool()`
- **Memory** enabled so it remembers context within a WhatsApp conversation thread
- A **RAI policy** to filter harmful or inappropriate messages

We use the customer's phone number as the `session_id` — this means each phone number gets
its own isolated conversation history automatically.

The agent's job is to look up orders and generate a response. The webhook handler (or the
simulation cell below) is responsible for calling `send_whatsapp_reply()` to deliver that
response to the customer.

In [ ]:
# Create the WhatsApp co-pilot agent
whatsapp_agent = studio.create_agent(
    name="WhatsApp Business Co-pilot",
    provider="openai/gpt-4o",
    role="Customer service representative for an e-commerce business",
    goal="Help customers with order inquiries, product questions, and support requests via WhatsApp",
    instructions=(
        "You are a friendly, concise customer service agent. "
        "Always look up order status when a customer provides an order ID. "
        "Keep replies under 300 characters (WhatsApp best practice). "
        "If a customer asks to speak to a human, acknowledge it and say a team member will follow up within 1 hour."
    )
)

# Attach tools — only lookup_order; send_whatsapp_reply is called by the webhook handler
whatsapp_agent.add_tool(lookup_order)

# Enable memory — keyed by session_id (customer phone number)
whatsapp_agent.add_memory(max_messages=10)

# RAI: filter harmful content
rai_policy = studio.create_rai_policy(
    name="WhatsApp RAI Policy",
    toxicity=True,
    pii="redact"
)
whatsapp_agent.add_rai_policy(rai_policy)

print(f"Agent ready: {whatsapp_agent.id}")

## 4. Simulating a Customer Conversation

In production, each inbound WhatsApp message triggers one call to your webhook, which calls
`agent.run(message, session_id=customer_phone)`. We simulate three turns of a real
customer journey:

1. **Product inquiry** — customer asks about a product
2. **Order status** — customer provides their order ID
3. **Escalation** — customer requests a human agent

The `session_id` is the customer's phone number. This ensures the agent remembers context
across all three turns — just as it would in a real WhatsApp thread.

In [ ]:
# Simulate a customer WhatsApp conversation
CUSTOMER_PHONE = "+1-555-0147"

print("=" * 60)
print(f"Customer: {CUSTOMER_PHONE}")
print("=" * 60)

# Turn 1: Product inquiry
print("\n[Customer]: Do you have wireless headphones? What's the price range?")
r1 = whatsapp_agent.run(
    "Do you have wireless headphones? What's the price range?",
    session_id=CUSTOMER_PHONE
)
send_whatsapp_reply(CUSTOMER_PHONE, r1.response)

# Turn 2: Order status lookup
print("\n[Customer]: I placed order ORD-1001 last week. Where is it?")
r2 = whatsapp_agent.run(
    "I placed order ORD-1001 last week. Where is it?",
    session_id=CUSTOMER_PHONE
)
send_whatsapp_reply(CUSTOMER_PHONE, r2.response)

# Turn 3: Escalation
print("\n[Customer]: I'd like to speak with a human agent please.")
r3 = whatsapp_agent.run(
    "I'd like to speak with a human agent please.",
    session_id=CUSTOMER_PHONE
)
send_whatsapp_reply(CUSTOMER_PHONE, r3.response)

## 5. Production Webhook (FastAPI)

In production, replace the direct `agent.run()` calls with this FastAPI webhook receiver.
Meta POSTs to `/webhook` every time a customer sends a WhatsApp message.

This code is **shown here for reference** — it is not run inline in this notebook (it requires
a public HTTPS URL for Meta to reach). Save it as `whatsapp_webhook.py` and run with
`uvicorn whatsapp_webhook:app --host 0.0.0.0 --port 8000`.

```python
# whatsapp_webhook.py — production webhook receiver
from fastapi import FastAPI, Request, HTTPException
from lyzr import Studio
import os

app = FastAPI()
studio = Studio(api_key=os.environ["LYZR_API_KEY"])

# Re-create agent (or load by ID: studio.get_agent("agent-id"))
agent = studio.create_agent(...)
agent.add_tool(lookup_order)
agent.add_memory(max_messages=10)

@app.get("/webhook")
async def verify(hub_mode: str, hub_challenge: str, hub_verify_token: str):
    """Meta webhook verification handshake."""
    if hub_verify_token == os.environ["WEBHOOK_VERIFY_TOKEN"]:
        return int(hub_challenge)
    raise HTTPException(status_code=403, detail="Invalid verify token")

@app.post("/webhook")
async def receive_message(request: Request):
    """Handle inbound WhatsApp messages."""
    body = await request.json()
    entry = body["entry"][0]["changes"][0]["value"]
    message = entry["messages"][0]
    from_number = message["from"]
    text = message["text"]["body"]

    # Use customer phone number as session ID for per-customer memory
    response = agent.run(text, session_id=from_number)
    send_whatsapp_reply(from_number, response.response)
    return {"status": "ok"}
```

## Common Mistake: Not Using `session_id`

The most critical mistake with WhatsApp integrations is **forgetting `session_id`** or
using a random one each turn. Without it, the agent starts fresh every message — it has
no idea what the customer said previously.

| Turn | Without `session_id` | With `session_id=phone` |
|---|---|---|
| Turn 1 | "Hi, my order is ORD-1001" | "Hi, my order is ORD-1001" |
| Turn 2 | "Where is it?" → ❌ "Where is what?" | "Where is it?" → ✅ looks up ORD-1001 |
| Turn 3 | "Cancel it" → ❌ confused | "Cancel it" → ✅ cancels ORD-1001 |

**Rule:** Always pass `session_id=customer_phone` to keep each customer's conversation coherent.

In [ ]:
# ❌ Mistake: different session_id each turn (or no session_id)
print("WITHOUT session_id (each turn is isolated):\n")

r1 = whatsapp_agent.run("Hi, my order is ORD-1001", session_id=str(uuid.uuid4()))
print(f"Turn 1: {r1.response[:100]}...")

r2 = whatsapp_agent.run("Where is it?", session_id=str(uuid.uuid4()))  # new session!
print(f"Turn 2: {r2.response[:100]}...")  # agent has no idea what "it" refers to

print("\n✅ Solution: use session_id=customer_phone for the entire conversation thread")

## Exercise: Add a "Track Shipment" Tool

Your task is to add a third tool to the WhatsApp agent that provides detailed tracking
information for a shipment.

**Steps:**

1. Write a `track_shipment(tracking_number: str) -> str` function with a clear docstring
2. Add simulated tracking events (e.g., "Package picked up", "In transit", "Out for delivery")
3. Attach the tool to `whatsapp_agent` with `whatsapp_agent.add_tool(track_shipment)`
4. Test it: ask the agent "Can you track FX123456 for me?"

Fill in the `...` placeholders below.

In [ ]:
# Simulated tracking database
TRACKING_EVENTS = {
    "FX123456": [
        "2026-02-23 09:00 — Package picked up at seller warehouse",
        "2026-02-23 18:30 — In transit to regional sort facility",
        "2026-02-24 06:00 — Arrived at regional sort facility",
        "2026-02-25 08:00 — Out for delivery",
    ],
    "UP789012": [
        "2026-02-24 10:00 — Package picked up",
        "2026-02-24 20:00 — Processing at origin facility",
    ],
}

# TODO: Write a track_shipment tool function with a clear docstring
def track_shipment(tracking_number: str) -> str:
    """..."""  # TODO: Write a clear docstring — this is what the agent reads!
    # TODO: Look up tracking_number in TRACKING_EVENTS
    # Return all events as a formatted string, or a helpful message if not found
    ...


# TODO: Add the tool to whatsapp_agent
# whatsapp_agent.add_tool(...)

# TODO: Test the agent
# response = whatsapp_agent.run("Can you track FX123456 for me?", session_id="+1-555-0147")
# print(response.response)

## Summary

### WhatsApp Business Co-pilot Architecture

| Component | Implementation |
|---|---|
| Inbound message | Meta webhook POST → FastAPI `/webhook` endpoint |
| Agent | `studio.create_agent()` with customer service role |
| Order lookup | `lookup_order(order_id)` tool — reads from DB/API |
| Reply | `send_whatsapp_reply(to, msg)` tool — calls Meta Graph API |
| Conversation memory | `session_id=customer_phone` — one session per customer |
| Safety | `studio.create_rai_policy(toxicity=True, pii="redact")` |

### Key Takeaways

- Use `session_id=customer_phone` — this is the unique identifier for each WhatsApp thread
- The agent-as-webhook pattern is: receive message → `agent.run(text, session_id=phone)` → tool sends reply
- Any business logic (order lookup, CRM queries, inventory checks) becomes a tool with a clear docstring
- Keep WhatsApp replies under 300 characters — set this expectation in `instructions`
- Simulation mode (print instead of call Meta) lets you develop and test without credentials

### Meta Cloud API Quick Reference

```python
# Production send (replaces the simulated send_whatsapp_reply)
import httpx

def send_whatsapp_reply(to_number: str, message: str) -> str:
    """Send a WhatsApp message via Meta Cloud API."""
    url = f"https://graph.facebook.com/v18.0/{PHONE_NUMBER_ID}/messages"
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}
    payload = {
        "messaging_product": "whatsapp",
        "to": to_number,
        "type": "text",
        "text": {"body": message}
    }
    r = httpx.post(url, json=payload, headers=headers)
    return f"Sent: {r.status_code}"
```

## Next Steps

You have completed Lesson 14. From here you can:

- **Lesson 15: Multi-agent E-commerce** — build a manager agent that routes customer
  queries to three specialist sub-agents using the agent-as-tool pattern

---

| Lesson | Topic |
|---|---|
| [01](./01_getting_started.ipynb) | Getting Started |
| [05](./05_memory_and_sessions.ipynb) | Memory & Sessions |
| [06](./06_tools_and_functions.ipynb) | Tools & Functions |
| [09](./09_rai_guardrails.ipynb) | RAI Guardrails |
| **14** | **WhatsApp Co-pilot (this lesson)** |
| [15](./15_multi_agent_ecommerce.ipynb) | Multi-agent E-commerce |